In [1]:
from transformers import BertModel,BertTokenizer
import torch
from torch.utils.data import Dataset,DataLoader
from datasets import load_from_disk

In [2]:
class BizDataset(Dataset):
    def __init__(self,split):
        self.dataset = load_from_disk(r"D:\Workspace\regression\huggingface\dataset")
        if split=="train":
            self.dataset=self.dataset["train"]
        elif split=="test":
            self.dataset=self.dataset["test"]
        elif split=="validation":
            self.dataset=self.dataset["validation"]
        else:
            print("split is wrong")
           
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,item):
        text = self.dataset[item]["text"]
        label = self.dataset[item]["label"]
        return text,label

In [3]:

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768,2)
    # 前向推理运算
    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,attention_mask= attention_mask,token_type_ids=token_type_ids)
        
        out= self.fc(out.last_hidden_state[:,0])
        out = out.softmax(dim=1)
        return out

In [4]:
# 对数据进行编码处理
def collate_fn(data):
    sentes = [i[0] for i in data]
    labels = [i[1] for i in data]
    data = tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=sentes,
        padding="max_length",  # 填充到最大长度
        truncation=True,       # 截断超过最大长度的序列
        max_length=30,         # 最大长度为 10
        return_length=True,
        return_tensors="pt",
    )
    input_ids = data["input_ids"]
    attention_mask = data["attention_mask"]
    token_type_ids = data["token_type_ids"]
    labels = torch.LongTensor(labels)
    
    return input_ids,attention_mask,token_type_ids,labels

In [5]:
DEVICE= torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_name="bert-base-chinese"
model_name=r"D:\Workspace\regression\huggingface\model\bert-base-chinese\models--bert-base-chinese\snapshots\c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f"
tokenizer = BertTokenizer.from_pretrained(model_name)
pretrained = BertModel.from_pretrained(model_name).to(DEVICE)

# 1、创建数据集
test_ds= BizDataset("test")
# 2、创建数据加载器
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=8,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)

In [6]:
if __name__=='__main__':
    acc=0
    total=0
    print(DEVICE)
    model=Model().to(DEVICE)
#     optimizer=AdamW(model.parameters(),lr=1e-3)    
#     loss_func =torch.nn.CrossEntropyLoss()
    model.load_state_dict(torch.load("params/2bert.pt"))
    model.eval()

    for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(test_loader):
        input_ids,attention_mask,token_type_ids,labels = input_ids.to(DEVICE),attention_mask.to(DEVICE),token_type_ids.to(DEVICE),labels.to(DEVICE)
        # 执行前向计算
        out= model(input_ids,attention_mask,token_type_ids)
 
        out = out.argmax(dim=1)
        acc += (out == labels).sum().item()
        total += len(labels)
        
    print(acc/total)    

cpu


C:\Users\ldz11\AppData\Local\Temp\ipykernel_33476\1024724338.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("params/2bert.pt"))


0.84
